# Spatio-Temporal simulation

<!-- SUMMARY: Simulations of a Spatio-Temporal phenomenon -->

<!-- CATEGORY: Spatio-Temporal -->

In [16]:
import numpy as np
import gstlearn as gl
import gstlearn.document as gdoc
from scipy.special import gamma
from IPython.display import Markdown

In [ ]:
Markdown(gdoc.loadDoc("Spatio_Temporal.md"))

In [18]:
from sksparse.cholmod import cholesky
import scipy as sc
from scipy.sparse import diags

In [19]:
scale = 4.
kappa = 1./scale
kappa2 = kappa**2
dt = .1
dx = 1. 
c = 40
sqc = np.sqrt(c)
sqdt = np.sqrt(dt)

In [ ]:
nx = [100,100]
m = gl.Model.createFromParam(gl.ECov.MATERN,range=1,param=2,flagRange=False)
mesh = gl.MeshETurbo(nx ,[dx,dx])
S = gl.ShiftOpMatrix(mesh,m.getCovAniso(0))
Smat = S.getS().toTL()
TildeC12 = diags(np.sqrt(S.getTildeC()))
invTildeC12 = diags(1./np.sqrt(S.getTildeC()))
G = TildeC12 @ Smat @ TildeC12
M = TildeC12 @ TildeC12
K = (kappa2 * M + G) @ (kappa2 * M + G) @ (kappa2 * M + G)
param = 3
P = M +  c*dt * K
cholP = cholesky(P)

def evalInvA(x) :
    return  cholP.solve_A(M @ x)

def evalInvB(x) :
    return sqc * sqdt * cholP.solve_A(TildeC12 @ x)

# TODO: Reason for the warning ??!
# https://stackoverflow.com/a/48273717

In [21]:
np.random.seed(123)
x = np.random.normal(size=Smat.shape[0])

for i in range(100):
    u = np.random.normal(size=Smat.shape[0])
    x = evalInvA(x) + evalInvB(u)

In [ ]:
anim = True

if anim :
    %matplotlib notebook
    import matplotlib.pyplot as plt
    from matplotlib.animation import FuncAnimation
    #x = np.random.normal(size=Smat.shape[0])
    xtot = np.zeros(shape = [Smat.shape[0],50])
    xtot[:,0] = x
    fig, ax = plt.subplots(figsize=(2,2))
    ln = plt.imshow(x.reshape(nx), 'BrBG')

def init():
    ln.set_data(x.reshape(nx))
    return ln

def update(frame):
    u = np.random.normal(size=Smat.shape[0])
    xtot[:,frame] = evalInvA(xtot[:,frame-1]) + evalInvB(u)
    ln.set_data(xtot[:,frame].reshape(nx))
    return ln

if anim :
    ani = FuncAnimation(fig, update, frames = 50,
                        init_func=init, blit=False,interval=10)
    p = plt.show()


In [23]:
%matplotlib inline

In [24]:
model = gl.Model.createFromParam(gl.ECov.MATERN,range = scale,param = param,flagRange=False)

rangev = 3.5 * model.getCovAniso(0).getRange()
h = np.linspace(0,rangev,100)
points = [gl.SpacePoint([0.,i]) for i in h]
cov = [model.eval(points[0],i) for i in points]

In [25]:
def correc(kappa,alpha,d=2):
    return gamma(alpha-d/2)/ (gamma(alpha)*(4*np.pi)**(d/2)*kappa**(2*alpha-d))

In [26]:
d = 2
param = 3
alpha = param+d/2
hmax = 3.5 * model.getCovAniso(0).getRange()
N = 2**8


ind = np.concatenate([np.arange(int(N/2),N-1,2),np.arange(0,int(N/2),2)])

a= np.pi * (N-1) / hmax
v = np.linspace(-1.,1.,N)
u = a/2 * v
deltau=a/(N-1)

xi = np.meshgrid(u,u)
normxi = np.array([i**2 + j**2 for i in u for j in u]).reshape((len(u),len(u)))

grxi = c * (kappa2 + normxi)**alpha * correc(kappa,param+d/2,d)

nres = 200
time = np.arange(0,nres,1) 
result = np.zeros(shape=[len(ind),nres])
for k in time :
    fourier = np.exp(-k *dt * np.abs(grxi))/(  1./c * (2*np.pi)**d*2*grxi)
    B = np.real(np.fft.fftn(fourier,norm="backward"))
    result[:,k] = B[0,:][ind]
    

In [ ]:
X= np.pi * v[np.arange(0,N,2)] /deltau 
points = [gl.SpacePoint([0.,i]) for i in X]
cov = [model.eval(gl.SpacePoint([0.,0]),i)/2. for i in points]
 

plt.plot(X,cov)
for i in range(10):
    covZ=result[:,i * 10]*deltau**d
    plt.plot(X, covZ)


lim = 1.5 * model.getCovAniso(0).getRange()
plt.xlim([-lim,lim])
a = plt.axvline(x=0.)

In [ ]:
i = plt.imshow(result)

In [ ]:
v

In [ ]:
print(result[:,0].max())